# Finetuning a Black-box Embedding Adapter

## Generate Corpus

In [1]:
import json

from llama_index import SimpleDirectoryReader
from llama_index.node_parser import SimpleNodeParser
from llama_index.schema import MetadataMode

In [2]:
TRAIN_FILES = ["../../../examples/data/10k/lyft_2021.pdf"]
VAL_FILES = ["../../../examples/data/10k/uber_2021.pdf"]

TRAIN_CORPUS_FPATH = "./data/train_corpus.json"
VAL_CORPUS_FPATH = "./data/val_corpus.json"

In [3]:
def load_corpus(files, verbose=False):
    if verbose:
        print(f"Loading files {files}")

    reader = SimpleDirectoryReader(input_files=files)
    docs = reader.load_data()
    if verbose:
        print(f"Loaded {len(docs)} docs")

    parser = SimpleNodeParser.from_defaults()
    nodes = parser.get_nodes_from_documents(docs, show_progress=verbose)

    if verbose:
        print(f"Parsed {len(nodes)} nodes")

    return nodes

We do a very naive train/val split by having the Lyft corpus as the train dataset, and the Uber corpus as the val dataset.

In [4]:
train_nodes = load_corpus(TRAIN_FILES, verbose=True)
val_nodes = load_corpus(VAL_FILES, verbose=True)

Loading files ['../../../examples/data/10k/lyft_2021.pdf']
Loaded 238 docs


Parsing documents into nodes:   0%|          | 0/238 [00:00<?, ?it/s]

Parsed 349 nodes
Loading files ['../../../examples/data/10k/uber_2021.pdf']
Loaded 307 docs


Parsing documents into nodes:   0%|          | 0/307 [00:00<?, ?it/s]

Parsed 418 nodes


### Generate synthetic queries

Now, we use an LLM (gpt-3.5-turbo) to generate questions using each text chunk in the corpus as context.

Each pair of (generated question, text chunk used as context) becomes a datapoint in the finetuning dataset (either for training or evaluation).

In [1]:
from llama_index.finetuning import (
    generate_qa_embedding_pairs,
    EmbeddingQAFinetuneDataset,
)

In [ ]:
train_dataset = generate_qa_embedding_pairs(train_nodes)
val_dataset = generate_qa_embedding_pairs(val_nodes)

train_dataset.save_json("train_dataset.json")
val_dataset.save_json("val_dataset.json")

In [2]:
# [Optional] Load
train_dataset = EmbeddingQAFinetuneDataset.from_json("train_dataset.json")
val_dataset = EmbeddingQAFinetuneDataset.from_json("val_dataset.json")

## Run Embedding Finetuning

In [11]:
from llama_index.finetuning import EmbeddingAdapterFinetuneEngine
from llama_index.embeddings import resolve_embed_model
import torch

base_embed_model = resolve_embed_model("local:BAAI/bge-small-en")

finetune_engine = EmbeddingAdapterFinetuneEngine(
    train_dataset,
    base_embed_model,
    # model_output_path="model_output_test",
    # model_output_path="model_output_test_v2",
    # model_output_path="model_output_test_bias",
    # model_output_path="model_output_no_bias_4",
    # model_output_path="model_output_no_bias_5",
    model_output_path="model_output_bias_6",
    bias=True,
    # epochs=2,
    epochs=4,
    verbose=True,
    # optimizer_class=torch.optim.SGD,
    # optimizer_params={"lr": 0.01}
)

In [ ]:
finetune_engine.finetune()

> Prepared optimizer, scheduler, and loss model.


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/67 [00:00<?, ?it/s]

> [Epoch 0] Current loss: 1.6461929082870483
> [Epoch 0] Current loss: 1.6520378589630127
> [Epoch 0] Current loss: 1.4205775260925293
> [Epoch 0] Current loss: 1.6287790536880493
> [Epoch 0] Current loss: 1.5413802862167358
> [Epoch 0] Current loss: 1.855971336364746
> [Epoch 0] Current loss: 1.8499667644500732
> [Epoch 0] Current loss: 1.5853407382965088
> [Epoch 0] Current loss: 1.6540038585662842
> [Epoch 0] Current loss: 1.6485378742218018
> [Epoch 0] Current loss: 1.6767240762710571
> [Epoch 0] Current loss: 1.7436771392822266
> [Epoch 0] Current loss: 1.5540363788604736
> [Epoch 0] Current loss: 1.6454252004623413
> [Epoch 0] Current loss: 1.59978449344635
> [Epoch 0] Current loss: 1.6202552318572998
> [Epoch 0] Current loss: 1.396388292312622
> [Epoch 0] Current loss: 1.6879438161849976
> [Epoch 0] Current loss: 1.490260124206543
> [Epoch 0] Current loss: 1.7628872394561768
> [Epoch 0] Current loss: 1.8766876459121704
> [Epoch 0] Current loss: 2.0071003437042236
> [Epoch 0] Cur

Iteration:   0%|          | 0/67 [00:00<?, ?it/s]

> [Epoch 1] Current loss: 1.6278995275497437
> [Epoch 1] Current loss: 1.637595772743225
> [Epoch 1] Current loss: 1.3984235525131226
> [Epoch 1] Current loss: 1.6125218868255615
> [Epoch 1] Current loss: 1.5274169445037842
> [Epoch 1] Current loss: 1.844879388809204
> [Epoch 1] Current loss: 1.8354038000106812
> [Epoch 1] Current loss: 1.5656654834747314
> [Epoch 1] Current loss: 1.633568525314331
> [Epoch 1] Current loss: 1.6286585330963135
> [Epoch 1] Current loss: 1.6574878692626953
> [Epoch 1] Current loss: 1.7275493144989014
> [Epoch 1] Current loss: 1.534127950668335
> [Epoch 1] Current loss: 1.620722770690918
> [Epoch 1] Current loss: 1.578970193862915
> [Epoch 1] Current loss: 1.5955476760864258
> [Epoch 1] Current loss: 1.3680717945098877
> [Epoch 1] Current loss: 1.670469045639038
> [Epoch 1] Current loss: 1.4625911712646484
> [Epoch 1] Current loss: 1.7457126379013062
> [Epoch 1] Current loss: 1.8609737157821655
> [Epoch 1] Current loss: 1.985285997390747
> [Epoch 1] Curren

Iteration:   0%|          | 0/67 [00:00<?, ?it/s]

> [Epoch 2] Current loss: 1.607826590538025
> [Epoch 2] Current loss: 1.6231151819229126
> [Epoch 2] Current loss: 1.3773081302642822
> [Epoch 2] Current loss: 1.596382737159729
> [Epoch 2] Current loss: 1.514119267463684
> [Epoch 2] Current loss: 1.8348331451416016
> [Epoch 2] Current loss: 1.8226051330566406
> [Epoch 2] Current loss: 1.5489269495010376
> [Epoch 2] Current loss: 1.6163933277130127
> [Epoch 2] Current loss: 1.6122478246688843
> [Epoch 2] Current loss: 1.6416947841644287
> [Epoch 2] Current loss: 1.7144181728363037
> [Epoch 2] Current loss: 1.51851224899292
> [Epoch 2] Current loss: 1.6015201807022095
> [Epoch 2] Current loss: 1.563701868057251


In [5]:
embed_model = finetune_engine.get_finetuned_model()

In [6]:
embed_model._adapter.linear.weight

Parameter containing:
tensor([[ 1.0012e+00, -1.5689e-04,  1.7100e-04,  ..., -6.1885e-05,
          6.6540e-04,  5.5957e-04],
        [-9.5280e-04,  1.0014e+00,  2.7861e-04,  ..., -6.4484e-04,
         -2.1343e-04,  6.3622e-04],
        [ 6.9218e-04,  4.7994e-04,  1.0013e+00,  ..., -4.8046e-05,
         -6.8439e-05, -1.4849e-04],
        ...,
        [-3.7138e-04, -5.1329e-04,  4.3320e-04,  ...,  1.0007e+00,
          3.4029e-04,  2.9252e-04],
        [ 5.8444e-04,  1.8534e-04,  1.4414e-04,  ..., -9.7170e-05,
          1.0009e+00,  3.1936e-04],
        [ 9.3196e-04,  4.7439e-04, -8.2419e-05,  ...,  8.3514e-04,
         -7.5721e-04,  1.0003e+00]], requires_grad=True)

## Evaluate Finetuned Model

In [7]:
from llama_index.embeddings import OpenAIEmbedding
from llama_index import ServiceContext, VectorStoreIndex
from llama_index.schema import TextNode
from tqdm.notebook import tqdm
import pandas as pd

from eval_utils import evaluate, display_results

In [15]:
ada = OpenAIEmbedding()
ada_val_results = evaluate(val_dataset, ada)

Generating embeddings:   0%|          | 0/395 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 790/790 [02:41<00:00,  4.88it/s]


In [16]:
display_results(["ada"], [ada_val_results])

,retrievers,hit_rate,mrr
0,ada,0.870886,0.729156


In [7]:
bge = "local:BAAI/bge-small-en"
bge_val_results = evaluate(val_dataset, bge)

Generating embeddings:   0%|          | 0/395 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 790/790 [00:23<00:00, 34.00it/s]


In [8]:
display_results(["bge"], [bge_val_results])

,retrievers,hit_rate,mrr
0,bge,0.787342,0.643038


In [36]:
ft_val_results = evaluate(val_dataset, embed_model)

Generating embeddings:   0%|          | 0/395 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 790/790 [00:21<00:00, 36.73it/s]


In [37]:
display_results(["ft"], [ft_val_results])

,retrievers,hit_rate,mrr
0,ft,0.792405,0.644241


In [24]:
# TMP (look at train results)

ada = OpenAIEmbedding()
ada_train_results = evaluate(train_dataset, ada)

Generating embeddings:   0%|          | 0/334 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 668/668 [02:06<00:00,  5.28it/s]


In [25]:
display_results(["ada"], [ada_train_results])

,retrievers,hit_rate,mrr
0,ada,0.883234,0.684107


In [23]:
ft_train_results = evaluate(train_dataset, embed_model)

Generating embeddings:   0%|          | 0/334 [00:00<?, ?it/s]

100%|██████████████████████████████████████| 668/668 [00:16<00:00, 39.60it/s]


In [24]:
display_results(["ft"], [ft_train_results])

,retrievers,hit_rate,mrr
0,ft,0.781437,0.619736


In [ ]:
## TMP: with epochs=4
ft_val_results_v2 = evaluate(val_dataset, embed_model)

In [46]:
display_results(["ft"], [ft_val_results_v2])

,retrievers,hit_rate,mrr
0,ft,0.797468,0.648945


In [10]:
## TMP: with bias term
ft_val_results_v3 = evaluate(val_dataset, embed_model)

Generating embeddings:   0%|          | 0/395 [00:00<?, ?it/s]

100%|██████████████████████████████████████| 790/790 [00:22<00:00, 35.91it/s]


In [11]:
display_results(["ft"], [ft_val_results_v3])

,retrievers,hit_rate,mrr
0,ft,0.787342,0.636751


In [13]:
## TMP: with no bias term
ft_val_results_v4 = evaluate(val_dataset, embed_model)

Generating embeddings:   0%|          | 0/395 [00:00<?, ?it/s]

100%|██████████████████████████████████████| 790/790 [00:22<00:00, 35.63it/s]


In [14]:
display_results(["ft"], [ft_val_results_v4])

,retrievers,hit_rate,mrr
0,ft,0.787342,0.636118


In [21]:
## TMP: with no bias term + 4 epochs
ft_val_results_v5 = evaluate(val_dataset, embed_model)

Generating embeddings:   0%|          | 0/395 [00:00<?, ?it/s]

100%|██████████████████████████████████████| 790/790 [00:20<00:00, 38.55it/s]


In [22]:
display_results(["ft"], [ft_val_results_v5])

,retrievers,hit_rate,mrr
0,ft,0.778481,0.631751


In [8]:
## TMP: with no bias term + 4 epochs (try again)
ft_val_results_v6 = evaluate(val_dataset, embed_model)

Generating embeddings:   0%|          | 0/395 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 790/790 [00:22<00:00, 35.74it/s]


In [9]:
display_results(["ft"], [ft_val_results_v6])

,retrievers,hit_rate,mrr
0,ft,0.8,0.661983
